<h1> Conditional Probability </h1>
<p> What a user will buy next </p>

In [2]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances
import numpy as np


item_names = ['Apples', 'Bananas', 'Bread', 'Jam', 'football','soap']
item_id_mapping = {item: i for i, item in enumerate(item_names)}
print(item_id_mapping)

user_item_matrix = [[1, 0, 3, 0, 2],
                   [4, 0, 0, 5, 1],
                   [0, 2, 0, 4, 0],
                   [3, 0, 0, 0, 5]]


user_item_matrix_with_ids = [[item_id_mapping.get(item, item) for item in row] for row in user_item_matrix]
user_similarity = 1 - pairwise_distances(user_item_matrix_with_ids, metric='cosine')

user_to_predict = 1
predicted_ratings = np.dot(user_similarity[user_to_predict, :], user_item_matrix) / np.sum(np.abs(user_similarity[user_to_predict, :]))
print(predicted_ratings)
top_recommendations = np.argsort(predicted_ratings)[::-1][:5]

print("Top Recommendations (Numerical IDs):", top_recommendations)
print("Top Recommendations (Item Names):", [item_names[i] for i in top_recommendations])


{'Apples': 0, 'Bananas': 1, 'Bread': 2, 'Jam': 3, 'football': 4, 'soap': 5}
[2.34443805 0.57809709 0.31093118 3.25054984 1.56834157]
Top Recommendations (Numerical IDs): [3 0 4 1 2]
Top Recommendations (Item Names): ['Jam', 'Apples', 'football', 'Bananas', 'Bread']


<h1> Joint Probability </h1>

Which items generally users buy together

In [3]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder  

shopping_cart = [['bread','milk','jam'],
               ['bread','butter','milk'],
               ['milk','butter','bread','jam'],
               ['butter','jam']]

te = TransactionEncoder()
df = pd.DataFrame(te.fit_transform(shopping_cart),columns=te.columns_)

# Calculate joint probabilities 
joint_probs = df.mean().to_dict()
print("Joint probabilities:", joint_probs)

# Calculate conditional probabilities
'''conditional_probs = {}

b_purchases = df['bread'].sum() # total baskets with bread

for item in df.columns:
   item_and_bread = (df[item] & df['bread']).sum() 
   prob = item_and_bread / b_purchases
   conditional_probs[item] = prob
'''
conditional_probs = {}

for antecedent_item in df.columns:

    antecedent_count = df[antecedent_item].sum() 
    
    for consequent_item in df.columns:
    
        if consequent_item != antecedent_item:
        
            joint_count = (df[antecedent_item] & df[consequent_item]).sum()  
            prob = joint_count / antecedent_count 
            conditional_probs[(consequent_item, antecedent_item)] = prob
            



Joint probabilities: {'bread': 0.75, 'butter': 0.75, 'jam': 0.75, 'milk': 0.75}


In [4]:
#print("Conditional probabilities:", conditional_probs)

# What I will buy next if I bought {x}
current_item = "jam" 
candidates = [(k,v) for k,v in conditional_probs.items()]
candidates.sort(key=lambda x: x[1], reverse=True) 

print("Recommended next items given", current_item + ":", candidates[0][0])

Recommended next items given jam: ('milk', 'bread')


<h1> Using Standard Apriori Algorithm </h1>

In [5]:
#!pip install mlxtend
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# dataset of items having association
shopping_cart = [['bread', 'milk', 'jam'],
                 ['bread', 'butter', 'jam'],
                 ['milk', 'butter', 'bread', 'jam'],
                 ['spinach', 'jam']]

te = TransactionEncoder()
one_hot_encoded = te.fit_transform(shopping_cart)
df = pd.DataFrame(one_hot_encoded, columns=te.columns_)


frequent_itemsets = apriori(df, min_support=0.5, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

# If I bought {x}  what I will buy next??
antecedent_item = 'jam'
matching_rules = rules[rules['antecedents'] == {antecedent_item}]

if not matching_rules.empty:
    recommended_items = matching_rules.iloc[0]['consequents']
    print("Recommended Items for the next purchase if user buys", antecedent_item, ":", recommended_items)
else:
    print("No rules found for predicting the next purchase based on", antecedent_item)


Recommended Items for the next purchase if user buys jam : frozenset({'bread'})
